# 🧪 **Template LCEN – Parkinson (Instalación → EDA → Preprocesamiento → Modelado)**

**Fecha:** 2025-09-25  
**Autor:** _Equipo LCEN_  
**Proyecto:** _ReMePARK / LARGE‑PD / PD GENEration (adaptar)_

Este cuaderno está personalizado para flujos del **LCEN–INNN** con columnas típicas en PD:
`id_paciente`, `sitio`, `fecha_visita`, `sexo`, `edad`, `grupo`, `mds_updrs_total`, `moca`, `pdq39_si`, `eq5d_index`.

Incluye:
- Paths estandarizados de proyecto
- Carga/validación de esquema
- EDA mínimo y automatizado
- Preprocesamiento (One-Hot + escalado, imputación)
- Modelos (sklearn + statsmodels OLS/Logit/Mixtos con `C(sitio)`)
- Guardado de artefactos (csv, figs, joblib)


##🗺️ **Índice del Cuaderno**

- [0) Configuración del proyecto](#0-configuracion-del-proyecto)
- [1) Importar librerías](#1-importar-librerias)
- [2) Esquema esperado del dataset (LCEN – PD)](#2-esquema-esperado-del-dataset-lcen-–-pd)
- [3) Carga de datos](#3-carga-de-datos)
- [4) Validación de esquema y tipos](#4-validacion-de-esquema-y-tipos)
- [5) EDA mínimo](#5-eda-minimo)
- [6) Visualizaciones básicas](#6-visualizaciones-basicas)
- [7) Definir TARGET, split y plan de preprocesamiento](#7-definir-target-split-y-plan-de-preprocesamiento)
- [8) Modelo ejemplo `sklearn`](#8-modelo-ejemplo-sklearn)
- [9) Modelos `statsmodels` con efectos fijos de sitio](#9-modelos-statsmodels-con-efectos-fijos-de-sitio)
- [10) Diagnósticos OLS y multicolinealidad](#10-diagnosticos-ols-y-multicolinealidad)
- [11) Guardado de artefactos](#11-guardado-de-artefactos)
- [12) Checklist LCEN (rápido)](#12-checklist-lcen-rapido)

# 🛠️**0) Configuración del proyecto**

**Importación de Librerías**

`pathlib`: Se importa la clase Path para manejar las rutas de archivos y carpetas de una manera más fácil y compatible con cualquier sistema operativo (Windows, Mac, Linux).

`warnings`: Permite controlar los mensajes de advertencia que Python pueda generar.

`random` y `numpy`: Se importan para trabajar con números aleatorios. Lo usarás para todo el cálculo matemático "pesado" detrás de pandas, para manejar valores nulos (np.nan) y para operaciones vectorizadas en las columnas mds_updrs o moca.

In [ ]:
from pathlib import Path
import warnings, random, numpy as np

**📂 Definición de la Estructura de Carpetas**

`PROJ_DIR = Path.cwd()`: Obtiene la ruta de la carpeta actual donde se está ejecutando el script (el directorio del proyecto).

Las siguientes líneas definen las rutas para subcarpetas importantes dentro del proyecto, como `data` (para guardar los datos), `outputs `(para guardar resultados), `figs` (para figuras o gráficos), `models` (para modelos guardados) y `reports` (para reportes). Usar `pathlib` permite unirlas de forma limpia con el operador `/`.

In [ ]:
PROJ_DIR = Path.cwd()
DATA_DIR = PROJ_DIR / "data"
OUTPUT_DIR = PROJ_DIR / "outputs"
FIG_DIR = OUTPUT_DIR / "figs"
MODEL_DIR = OUTPUT_DIR / "models"
REPORTS_DIR = OUTPUT_DIR / "reports"



Proyecto: /content


**🏗️ Creación Automática de las Carpetas**

Este es un bucle que recorre todas las rutas definidas anteriormente y crea las carpetas si no existen.

`p.mkdir()`: Es el comando para crear una nueva carpeta.

`parents=True`: Asegura que se creen también las carpetas "padre" si es necesario. Por ejemplo, si `outputs` no existe, la creará antes de intentar crear `figs` dentro de ella.

`exist_ok=True`: Evita que el programa dé un error si la carpeta ya existe. Simplemente no hace nada.

In [ ]:
for p in [DATA_DIR, OUTPUT_DIR, FIG_DIR, MODEL_DIR, REPORTS_DIR]:
    p.mkdir(parents=True, exist_ok=True)

**🎲 Configuración para la Reproducibilidad**

`SEED` = 42: Se define una "semilla" (seed) con un número fijo (42 es una elección popular por tradición).

`random.seed(SEED)` y `np.random.seed(SEED)`: Se establece esta semilla para las librerías `random` y `numpy`. Esto es **fundamental para la reproducibilidad**. Al fijar la semilla, cualquier proceso que involucre aleatoriedad (como dividir datos o inicializar un modelo) producirá **exactamente los mismos resultados** cada vez que se ejecute el código.

In [ ]:
SEED = 42
random.seed(SEED); np.random.seed(SEED)
warnings.filterwarnings("ignore")

print("Proyecto:", PROJ_DIR)

Proyecto: /content


#🐍 **1) Importar librerías**

**🧱 Librerías Fundamentales**

`sys` y `platform`: Módulos para interactuar con el sistema y obtener información sobre el entorno de ejecución, como la versión de Python o el sistema operativo.

`numpy`: Es la librería principal para el cálculo numérico. Introduce los arrays, que son estructuras de datos muy eficientes para operaciones matemáticas con grandes volúmenes de números.

`pandas`: Fundamental para la manipulación y análisis de datos. Su estructura principal, el DataFrame, es una tabla similar a una hoja de cálculo o una tabla de SQL, pero con funcionalidades muy potentes.

`matplotlib.pyplot`: Es la librería más utilizada para la visualización de datos, permitiendo crear todo tipo de gráficos y figuras.

In [ ]:
import sys, platform
import numpy as np, pandas as pd
import matplotlib.pyplot as plt

**🧹 Preparación y Preprocesamiento de Datos**

`train_test_split`: Divide un conjunto de datos en dos: uno para entrenar el modelo y otro para probar su rendimiento.

`StratifiedKFold` y `cross_val_score`: Herramientas para la validación cruzada, una técnica robusta para evaluar el rendimiento de un modelo dividiendo los datos en múltiples pliegues (folds) y promediando los resultados.

`OneHotEncoder`: Convierte variables categóricas (ej: "rojo", "verde") en un formato numérico que los modelos puedan entender (ej: columnas binarias [1, 0], [0, 1]).

`StandardScaler`: Estandariza variables numéricas para que tengan una media de 0 y una desviación estándar de 1. Esto es crucial para muchos algoritmos.

`SimpleImputer`: Sirve para manejar valores faltantes (nulos) en los datos, por ejemplo, rellenándolos con la media o la mediana de la columna.

`ColumnTransformer` y `Pipeline`: Permiten organizar y encadenar todos los pasos de preprocesamiento en un flujo de trabajo ordenado y reproducible.

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

**🤖 Modelado y Evaluación**

`LogisticRegression`: Un modelo de clasificación para predecir una categoría (ej: "sí/no", "compra/no compra").

`LinearRegression`: Un modelo de regresión para predecir un valor numérico continuo (ej: el precio de una casa).

`classification_report`, `confusion_matrix`, `roc_auc_score`: Métricas para evaluar modelos de clasificación.

`mean_squared_error`: Una métrica común para evaluar modelos de regresión.

In [ ]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, mean_squared_error

**📊 Análisis Estadístico y Diagnóstico**

`sm` y `smf`: Módulos para crear modelos estadísticos (como regresiones lineales o logísticas) y obtener información detallada como p-valores, intervalos de confianza, etc.

`anova_lm`: Realiza una tabla ANOVA para comparar modelos.

`variance_inflation_factor` (VIF): Mide la multicolinealidad (qué tan correlacionadas están las variables predictoras entre sí).

`het_breuschpagan`, `het_white`: Pruebas para detectar heterocedasticidad (si la varianza de los errores no es constante).

`acorr_ljungbox`: Prueba para detectar autocorrelación en los errores, común en series de tiempo.

`jarque_bera`: Prueba si los errores del modelo siguen una distribución normal.

In [ ]:
# Statsmodels (para OLS/Logit/Mixtos/ANOVA/diagnósticos)
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan, het_white, acorr_ljungbox
from statsmodels.stats.stattools import jarque_bera

**✅ Verificación del Entorno**

Muestra la versión de Python, el sistema operativo y la versión de Pandas que se están utilizando. Esto es clave para la reproducibilidad y para compartir el trabajo con otros.

In [ ]:
print("Python:", sys.version.split()[0])
print("Platform:", platform.platform())
print("Pandas:", pd.__version__)

Python: 3.12.11
Platform: Linux-6.6.97+-x86_64-with-glibc2.35
Pandas: 2.2.2


#📋 **2) Esquema esperado del dataset (LCEN – PD)**
Ajusta según tu estudio. Estas columnas son comunes en proyectos del LCEN.


En Python, casi todo es un **objeto**. Piensa en un objeto como una "cosa" digital que tiene tanto datos (atributos) como acciones que puede realizar (métodos). Una **clase** es el plano o molde para crear estos objetos.

Las comillas (simples `'` o dobles `"`) se usan para crear un tipo de objeto fundamental: los strings, que simplemente son texto. El texto dentro de las comillas es el dato que contiene el objeto string.

El símbolo de gato o almohadilla (`#`) le dice al intérprete de Python: "Ignora todo lo que venga después de mí en esta línea". No es código que se ejecuta, es texto exclusivo para los humanos.

`[]` - Corchetes (Listas y Acceso)
Los corchetes se usan principalmente para dos cosas: crear listas y acceder a elementos.

`()` - Paréntesis (Tuplas y Llamadas a Funciones)
Los paréntesis tienen tres usos principales: llamar funciones, crear tuplas y agrupar operaciones matemáticas.

`{}` - Llaves (Diccionarios y Conjuntos)
Las llaves se utilizan para crear diccionarios y conjuntos.

In [ ]:
expected_cols = [
    "id_paciente",     # identificador único
    "sitio",           # centro/país/ciudad
    "fecha_visita",    # fecha de evaluación
    "sexo",            # 'M'/'F' (estandarizar)
    "edad",            # en años
    "grupo",           # 'control'/'caso' (o 'PD'/'NoPD')
    "mds_updrs_total", # continuo
    "moca",            # 0-30
    "pdq39_si",        # summary index (0-100) si disponible
    "eq5d_index"       # -0.59 a 1 (según sistema)
]

##📥 **3) Carga de datos**

**🚫 Carga de Datos (Actualmente Desactivada)**

La primera línea intentaría leer un archivo CSV (`.csv`).

La segunda línea intentaría leer un archivo Excel (`.xlsx`)

In [ ]:
# df = pd.read_csv(DATA_DIR / "lcen_pd_base.csv", encoding="utf-8", na_values=["", "NA", "NaN"])
# df = pd.read_excel(DATA_DIR / "lcen_pd_base.xlsx", sheet_name=0)


**🧪 El Plan B: Creación de Datos de Demostración**

El código comprueba si los datos se cargaron y, si no, los inventa.

`if 'df' not in globals()`:: Esta es la condición clave. Se traduce como: "Si una variable llamada df no existe todavía en la memoria...". Como las líneas anteriores estaban comentadas, esta condición es verdadera y el código de adentro se ejecuta.

`rng = np.random.default_rng(0)`: Inicia un generador de números aleatorios. Poner un 0 como semilla asegura que siempre se generen los mismos números "aleatorios", lo que hace el resultado reproducible.

`df = pd.DataFrame({...})`: Aquí se crea un DataFrame de pandas desde cero. La estructura {} contiene un diccionario donde las claves son los nombres de las columnas y los valores son los datos generados para esas columnas.

In [ ]:
# DEMO si no hay archivo:
if 'df' not in globals():
    rng = np.random.default_rng(0)
    n = 600
    df = pd.DataFrame({
        "id_paciente": np.arange(1, n+1),
        "sitio": rng.choice(["INNN","Cleveland","Santiago","Bogotá"], size=n, p=[0.5,0.2,0.2,0.1]),
        "fecha_visita": pd.date_range("2024-01-01", periods=n, freq="D"),
        "sexo": rng.choice(["M","F"], size=n),
        "edad": rng.integers(35, 88, size=n),
        "grupo": rng.choice(["control","caso"], size=n, p=[0.4,0.6]),
        "mds_updrs_total": rng.normal(45, 15, size=n).round(1),
        "moca": rng.integers(10, 30, size=n),
        "pdq39_si": np.clip(rng.normal(35, 18, size=n), 0, 100).round(1),
        "eq5d_index": np.clip(rng.normal(0.78, 0.15, size=n), -0.59, 1).round(3)
    })
df.head()

,id_paciente,sitio,fecha_visita,sexo,edad,grupo,mds_updrs_total,moca,pdq39_si,eq5d_index
0,1,Cleveland,2024-01-01,F,47,control,25.9,15,32.3,0.858
1,2,INNN,2024-01-02,M,84,control,35.1,10,37.4,0.412
2,3,INNN,2024-01-03,M,78,caso,41.6,15,13.6,0.542
3,4,INNN,2024-01-04,F,64,control,35.2,10,8.0,0.785
4,5,Santiago,2024-01-05,F,57,caso,41.4,20,32.6,0.656


#📐 **4) Validación de esquema y tipos**

**🔎 Verificación de Columnas**

Este bloque actúa como un guardia de seguridad para la estructura de tu tabla.

`missing`: Compara una lista predefinida de columnas esperadas (expected_cols) con las columnas que realmente tiene tu DataFrame. Si una columna esperada no está, la añade a la lista missing.

`extras`: Hace lo contrario. Revisa las columnas de tu DataFrame y si encuentra alguna que no estaba en la lista de esperadas, la añade a la lista `extras`.

`print(...)`: Finalmente, te informa si te faltan columnas cruciales o si tienes columnas extra que quizás no necesites.

In [ ]:
# Columnas faltantes/sobrantes
missing = [c for c in expected_cols if c not in df.columns]
extras = [c for c in df.columns if c not in expected_cols]
print("Faltantes:", missing)
print("Extras:", extras[:10])

Faltantes: []
Extras: []


**✨ Limpieza y Estandarización de Datos**

Este bloque se encarga de corregir los tipos de datos y estandarizar los valores dentro de las columnas.

`pd.to_datetime(...)`: Convierte la columna `fecha_visita` a un formato de fecha real.

`errors='coerce'`: Esta es una red de seguridad. Si encuentra un valor que no puede convertir a fecha (ej: "No aplica" o un error tipográfico), no detiene el programa, sino que lo convierte en `NaT` (Not a Time), un valor nulo especial para fechas.

`.str.strip()`: Elimina espacios en blanco al inicio y al final (ej: " `" M "`  -> `"M"`).

`.str.lower()`: Convierte todo a minúsculas (ej: `"Masculino"` -> `"masculino"`).

`.replace({...})`: Reemplaza valores para unificarlos (ej: `"femenino"` se convierte en `"f"`).

`.str.upper()`: Convierte el resultado final a mayúsculas.

In [ ]:
# Tipos y parsing
df['fecha_visita'] = pd.to_datetime(df['fecha_visita'], errors='coerce')
df['sexo'] = (df['sexo'].astype(str)
              .str.strip().str.lower()
              .replace({'femenino':'f','masculino':'m','m':'M','f':'F'})
              .str.upper())

**🩺 Validación de Valores (Sanity Checks)**

Este último bloque verifica que los datos sean lógicos y se encuentren dentro de rangos plausibles.

`assert` es una regla que debe cumplirse. Si la condición es verdadera, el código continúa sin hacer nada. Si la condición es falsa, el programa se detiene y muestra el mensaje de error.
El `print("Validación básica OK...")` al final es un mensaje de éxito. Solo lo verás si todas las aserciones `(assert)` anteriores se cumplieron.

El `for in` es una de las estructuras más importantes y usadas en Python. Su propósito es repetir un bloque de código una vez por cada elemento dentro de una secuencia (como una lista, un texto o un rango de números).

En pocas palabras, te permite "recorrer" o "iterar" sobre una colección de cosas y hacer algo con cada una de ellas.

In [ ]:
# Rango/valores plausibles (ajustar reglas si aplica)
assert df['edad'].between(18, 110).all(), "Revisa valores de edad"
for col in ['moca']:
    assert df[col].between(0, 30).all(), "MOCA fuera de rango (0-30)"
print("Validación básica OK (si no hay errores arriba).")

Validación básica OK (si no hay errores arriba).


##📊 **5) EDA mínimo**

In [ ]:
print("Dimensiones:", df.shape)
display(df.head(10))
display(df.describe(include='all'))

# NA y duplicados
na_counts = df.isna().sum().sort_values(ascending=False)
display(na_counts[na_counts>0])
print("Duplicados:", df.duplicated(subset=['id_paciente','fecha_visita']).sum())

Dimensiones: (600, 10)


,id_paciente,sitio,fecha_visita,sexo,edad,grupo,mds_updrs_total,moca,pdq39_si,eq5d_index
0,1,Cleveland,2024-01-01,F,47,control,25.9,15,32.3,0.858
1,2,INNN,2024-01-02,M,84,control,35.1,10,37.4,0.412
2,3,INNN,2024-01-03,M,78,caso,41.6,15,13.6,0.542
3,4,INNN,2024-01-04,F,64,control,35.2,10,8.0,0.785
4,5,Santiago,2024-01-05,F,57,caso,41.4,20,32.6,0.656
5,6,Bogotá,2024-01-06,F,83,caso,59.6,18,27.5,1.000
6,7,Cleveland,2024-01-07,M,58,caso,25.8,15,38.6,0.800
7,8,Santiago,2024-01-08,F,52,control,27.1,16,59.4,0.972
8,9,Cleveland,2024-01-09,F,61,control,45.7,20,25.8,0.798
9,10,Bogotá,2024-01-10,F,75,control,36.5,24,22.7,0.805


,id_paciente,sitio,fecha_visita,sexo,edad,grupo,mds_updrs_total,moca,pdq39_si,eq5d_index
count,600.000000,600,600,600,600.000000,600,600.000000,600.000000,600.000000,600.000000
unique,NaN,4,NaN,2,NaN,2,NaN,NaN,NaN,NaN
top,NaN,INNN,NaN,F,NaN,caso,NaN,NaN,NaN,NaN
freq,NaN,274,NaN,302,NaN,345,NaN,NaN,NaN,NaN
mean,300.500000,NaN,2024-10-26 12:00:00,NaN,61.471667,NaN,44.242500,19.223333,35.122000,0.776242
min,1.000000,NaN,2024-01-01 00:00:00,NaN,35.000000,NaN,7.400000,10.000000,0.000000,0.365000
25%,150.750000,NaN,2024-05-29 18:00:00,NaN,48.000000,NaN,33.875000,15.000000,23.100000,0.677000
50%,300.500000,NaN,2024-10-26 12:00:00,NaN,62.000000,NaN,43.100000,19.000000,35.850000,0.778500
75%,450.250000,NaN,2025-03-25 06:00:00,NaN,76.000000,NaN,54.825000,24.000000,46.750000,0.879250
max,600.000000,NaN,2025-08-22 00:00:00,NaN,87.000000,NaN,84.900000,29.000000,88.500000,1.000000


,0


Duplicados: 0


##📈 **6) Visualizaciones básicas**

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
df['edad'].hist(bins=25)
plt.title('Distribución de edad'); plt.xlabel('Edad'); plt.ylabel('Frecuencia')
plt.show()

plt.figure()
df.boxplot(column='mds_updrs_total', by='grupo')
plt.title('MDS‑UPDRS total por grupo'); plt.suptitle(''); plt.xlabel('Grupo'); plt.ylabel('UPDRS')
plt.show()

##🎯 **7) Definir TARGET, ✂️ split y 🗺️ plan de preprocesamiento**

In [ ]:
TARGET = 'grupo'  # 'grupo' (control/caso) o usa 'mds_updrs_total'/'moca' si es regresión
y = df[TARGET]
X = df.drop(columns=[TARGET])

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_cols = X.select_dtypes(include=np.number).columns.tolist()
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()

numeric_pipe = Pipeline([('imputer', SimpleImputer(strategy='median')),
                         ('scaler', StandardScaler())])
categorical_pipe = Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),
                             ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

preprocess = ColumnTransformer([('num', numeric_pipe, num_cols),
                                ('cat', categorical_pipe, cat_cols)])

from sklearn.model_selection import train_test_split
is_classification = y.nunique() <= 20
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y if is_classification else None
)

X_train.shape, X_test.shape, is_classification

##🧠 **8) Modelo ejemplo `sklearn`**

In [ ]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import classification_report, mean_squared_error

if is_classification:
    model = Pipeline([('preprocess', preprocess),
                      ('estimator', LogisticRegression(max_iter=500))])
else:
    model = Pipeline([('preprocess', preprocess),
                      ('estimator', LinearRegression())])

model.fit(X_train, y_train)
preds = model.predict(X_test)

if is_classification:
    print(classification_report(y_test, preds))
else:
    print("MSE:", mean_squared_error(y_test, preds))

##📉 **9) Modelos `statsmodels` con efectos fijos de sitio**

In [ ]:
df_sm = df.copy()

# OLS: outcome continuo (ej. mds_updrs_total) con efectos fijos por sitio
ols = smf.ols('mds_updrs_total ~ edad + C(sexo) + C(sitio)', data=df_sm).fit()
print(ols.summary())

# Logit: outcome binario (ej. grupo==caso)
df_sm['es_caso'] = (df_sm['grupo']=='caso').astype(int)
logit = smf.logit('es_caso ~ edad + C(sexo) + C(sitio)', data=df_sm).fit(disp=False)
print(logit.summary())

# MixedLM: medidas repetidas por paciente (si múltiples visitas)
if df_sm.duplicated('id_paciente').any():
    mixed = smf.mixedlm('mds_updrs_total ~ edad + C(sexo)', data=df_sm, groups=df_sm['id_paciente'])
    mixed_fit = mixed.fit(reml=True)
    print(mixed_fit.summary())

##🚦 **10) Diagnósticos OLS y multicolinealidad**

In [ ]:
resid = ols.resid; fitted = ols.fittedvalues

fig = sm.qqplot(resid, line='45')
plt.title('QQ-plot residuales OLS'); plt.show()

plt.figure(); plt.scatter(fitted, resid, alpha=0.6)
plt.axhline(0, ls='--'); plt.xlabel('Ajustados'); plt.ylabel('Residuales')
plt.title('Residuales vs Ajustados'); plt.show()

# Normalidad
jb_stat, jb_p, *_ = jarque_bera(resid)
print('Jarque–Bera p=', jb_p)

# Heterocedasticidad
bp_stat, bp_p, *_ = het_breuschpagan(resid, ols.model.exog)
w_stat, w_p, *_ = het_white(resid, ols.model.exog)
print('Breusch–Pagan p=', bp_p, '| White p=', w_p)

# VIF (sin intercepto duplicado)
from patsy import dmatrices
y_mat, X_mat = dmatrices('mds_updrs_total ~ edad + C(sexo) + C(sitio)', data=df_sm, return_type='dataframe')
if 'Intercept' in X_mat.columns: X_vif = X_mat.drop(columns=['Intercept'])
else: X_vif = X_mat.copy()
vif = pd.Series([variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])],
                index=X_vif.columns, name='VIF').sort_values(ascending=False)
print(vif)

##💾 **11) Guardado de artefactos**

In [ ]:
from joblib import dump

# Predicciones
preds_df = pd.DataFrame({
    "id_paciente": X_test.get("id_paciente", pd.Series(range(len(X_test)))),
    "y_true": y_test.values,
    "y_pred": preds
})
preds_path = OUTPUT_DIR / "predicciones_test.csv"
preds_df.to_csv(preds_path, index=False)

# Modelo sklearn
dump(model, MODEL_DIR / "pipeline_model.joblib")

# Tablas OLS/LOGIT a CSV rápidos
ols_params = ols.summary2().tables[1]
ols_params.to_csv(REPORTS_DIR / "ols_params.csv")
try:
    logit_params = logit.summary2().tables[1]
    logit_params.to_csv(REPORTS_DIR / "logit_params.csv")
except Exception as e:
    print("No se exportó logit:", e)

print("Guardado en:", OUTPUT_DIR)

##📝 **12) Checklist LCEN (rápido)**

- [ ] Validar codificación de `grupo` y definir claramente `TARGET`/outcome.
- [ ] Confirmar `sitio` (niveles consistentes) y considerar **SE robustos por clúster** (sitio).
- [ ] Si hay longitudinal: usar **MixedLM** con `groups=id_paciente` y (opcional) `re_formula`.
- [ ] Reportar **IC95%** y medidas de ajuste (AIC/BIC).
- [ ] Exportar reportes y figuras a `outputs/` con nombres versionados.
